In [2]:
%load_ext autoreload

In [3]:
import pandas as pd
from tqdm import tqdm
import torch
import sys
import os
import time
import datetime
import sys

In [4]:
import logging
logger = logging.getLogger("molecules")
logger.setLevel(logging.INFO)

fh = logging.FileHandler('logs.log')
fh.setLevel(logging.INFO)
logger.addHandler(fh)
logger.addHandler(logging.StreamHandler(stream=sys.stdout))

In [5]:
logger.info("asd")

asd


In [6]:
%autoreload 2
from huggingmolecules import MatConfig, MatFeaturizer, MatModel
from huggingmolecules import GroverConfig, GroverFeaturizer, GroverModel
from huggingmolecules import RMatConfig, RMatFeaturizer, RMatModel
from models import StrippedMatModel, StrippedGroverModel, StrippedRMatModel

In [13]:
%autoreload 2
from utils import load_ogb_dataset, load_tdc_dataset, featurize_with, get_huggingmolecules_processors, process

In [8]:
import tdc

In [9]:
import rdkit.rdBase as rkrb
import rdkit.RDLogger as rkl

logger = rkl.logger()
logger.setLevel(rkl.ERROR)
rkrb.DisableLog('rdApp.error')

In [11]:
ogb_datasets = [
    "ogbg-moltox21",
    "ogbg-molbace",
    "ogbg-molbbbp",
    "ogbg-molclintox",
    "ogbg-molmuv",
    "ogbg-molsider",
    "ogbg-moltoxcast",
    "ogbg-molesol",
    "ogbg-molfreesolv",
    "ogbg-mollipo"
]

In [12]:
logger = logging.getLogger("molecules")

logger.info(f"")
logger.info(f"Starting new procedure ({datetime.datetime.now()})")

if not os.path.exists("embeddings"):
    os.mkdir("embeddings")

for dataset_name in ogb_datasets:

    df, split_idx = load_ogb_dataset(dataset_name)

    if not os.path.exists(f"embeddings/{dataset_name}"):
        os.mkdir(f"embeddings/{dataset_name}")

    for config, model, featurizer in get_huggingmolecules_processors():

        logger.info(f"Processing {dataset_name} with {config}")
        
        smiles = df.copy(deep=True)

        if not os.path.isfile(f"embeddings/{dataset_name}/{config}.zip"):

            start = time.time()

            process(smiles, model, featurizer)

            smiles.to_csv(
                f'embeddings/{dataset_name}/{config}.zip',
                compression={"method": 'zip', "archive_name": f'embeddings.csv'},
                index=True
            )

            logger.info(f"Processing took {((time.time() - start)/60):.1f}m")

        else:
            logger.info("✔️")


Starting new procedure (2024-01-08 18:25:05.161535)


Processing ogbg-moltox21 with mat_masking_20M


  3%|▎         | 218/7831 [00:29<04:38, 27.37it/s]  

Heavy processing (166.3s) of 'C=C1C[C@]23CC[C@@H]4[C@](C)(C(=O)O[C@@H]5O[C@H](CO)[C@@H](O)[C@H](O)[C@H]5O)CCC[C@]4(C)[C@@H]2C[C@@H](O[C@@H]2O[C@H](CO)[C@@H](O)[C@H](O[C@@H]4O[C@H](CO)[C@@H](O)[C@H](O)[C@H]4O)[C@H]2OC2O[C@H](CO)[C@@H](O)[C@H](O)[C@H]2O)[C@@H]1C3'                    


 20%|██        | 1598/7831 [05:30<07:14, 14.34it/s]  

...

Processing took 45.5m
Processing ogbg-moltox21 with mat_masking_20M
✔️
Processing ogbg-moltox21 with mat_masking_20M
✔️


Downloading...
From (uriginal): https://drive.google.com/uc?id=1eQfPPWghmaq-dvWNO-bhom1UsovKn2h5
From (redirected): https://drive.google.com/uc?id=1eQfPPWghmaq-dvWNO-bhom1UsovKn2h5&confirm=t&uuid=e5d71982-8d0b-481d-a7ac-6e79c6aaf2eb
To: /home/michlampert/.cache/torch/huggingmolecules/grover_large.pt
100%|██████████| 429M/429M [02:32<00:00, 2.81MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1T2ot3tF6aRR9jsveKB5ZEdvCOpt-D8Aj
To: /home/michlampert/.cache/torch/huggingmolecules/grover_large.json
100%|██████████| 602/602 [00:00<00:00, 2.50MB/s]


Processing ogbg-moltox21 with grover_large


INFO:molecules:Processing ogbg-moltox21 with grover_large
 52%|█████▏    | 4052/7831 [00:15<00:14, 264.70it/s]


KeyboardInterrupt: 

In [14]:
pd.read_csv('embeddings/ogbg-moltox21/mat_masking_20M.zip', index_col=0)

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,smiles,features,embeddings
0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,"MatBatchEncoding(node_features=tensor([[[1., 0...","tensor([ 0.9850, -1.6232, -1.3696, ..., -0.45..."
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,CCN1C(=O)NC(c2ccccc2)C1=O,"MatBatchEncoding(node_features=tensor([[[1., 0...","tensor([ 1.1288, -1.6356, -1.2447, ..., -0.57..."
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...,"MatBatchEncoding(node_features=tensor([[[1., 0...","tensor([ 0.8760, -1.6521, -1.3986, ..., -0.48..."
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C,"MatBatchEncoding(node_features=tensor([[[1., 0...","tensor([ 0.9150, -1.5352, -1.3837, ..., -0.49..."
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CC(O)(P(=O)(O)O)P(=O)(O)O,"MatBatchEncoding(node_features=tensor([[[1., 0...","tensor([ 1.1751, -2.0371, -0.7114, ..., -0.45..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,CCOc1nc2cccc(C(=O)O)c2n1Cc1ccc(-c2ccccc2-c2nnn...,"MatBatchEncoding(node_features=tensor([[[1., 0...","tensor([ 1.2462, -1.3651, -1.4393, ..., -0.59..."
7827,1.0,1.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...,"MatBatchEncoding(node_features=tensor([[[1., 0...","tensor([ 0.6939, -1.7255, -1.3494, ..., -0.43..."
7828,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...,"MatBatchEncoding(node_features=tensor([[[1., 0...","tensor([ 0.8341, -1.6769, -1.3438, ..., -0.49..."
7829,1.0,1.0,0.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CC[C...,"MatBatchEncoding(node_features=tensor([[[1., 0...","tensor([ 0.9864, -1.5925, -1.4755, ..., -0.60..."


In [15]:
from tdc.single_pred import ADME

tdc_datasets = [
    (ADME, 'Caco2_Wang'),
    (ADME, 'Half_Life_Obach'),
]

In [18]:
logger.info(f"")
logger.info(f"Starting new procedure ({datetime.datetime.now()})")

if not os.path.exists("embeddings"):
    os.mkdir("embeddings")

for dataset_module, dataset_name in tdc_datasets:

    df, split_idx = load_tdc_dataset(dataset_module, dataset_name)

    if not os.path.exists(f"embeddings/{dataset_name}"):
        os.mkdir(f"embeddings/{dataset_name}")

    for config, model, featurizer in get_huggingmolecules_processors():

        logger.info(f"Processing {dataset_name} with {config}")
        
        smiles = df.copy(deep=True)

        if not os.path.isfile(f"embeddings/{dataset_name}/{config}.zip"):

            start = time.time()

            process(smiles, model, featurizer)

            smiles.to_csv(
                f'embeddings/{dataset_name}/{config}.zip',
                compression={"method": 'zip', "archive_name": f'embeddings.csv'},
                index=True
            )

            logger.info(f"Processing took {((time.time() - start)/60):.1f}m")

        else:
            logger.info("✔️")

INFO:molecules:


Starting new procedure (2024-01-08 19:21:08.539002)


INFO:molecules:Starting new procedure (2024-01-08 19:21:08.539002)
Found local copy...
Loading...
Done!


Processing Caco2_Wang with mat_masking_20M


INFO:molecules:Processing Caco2_Wang with mat_masking_20M
  9%|▉         | 85/910 [00:16<02:24,  5.72it/s]

...

Heavy processing (259.4s) of 'CN1[C@@H]2CC[C@@H]1CC(OC(=O)C(CO)c1ccccc1)C2'                    
 66%|██████▋   | 604/910 [11:36<03:29,  1.46it/s]  

Heavy processing (419.4s) of 'CN1CC[C@]23C(=O)C[C@@H]4C(=CCO[C@H]5CC(=O)N(c6ccccc62)[C@@H]3[C@@H]54)C1'                    


Heavy processing (419.4s) of 'CN1CC[C@]23C(=O)C[C@@H]4C(=CCO[C@H]5CC(=O)N(c6ccccc62)[C@@H]3[C@@H]54)C1'                    
 72%|███████▏  | 657/910 [19:06<00:48,  5.23it/s]    

Heavy processing (261.7s) of 'CC[C@H]1[C@@H]2C[C@@H]3[C@@H]4N(C)c5ccccc5[C@]45C[C@@H](C2[C@@H]5O)N3[C@@H]1O'                    


Heavy processing (261.7s) of 'CC[C@H]1[C@@H]2C[C@@H]3[C@@H]4N(C)c5ccccc5[C@]45C[C@@H](C2[C@@H]5O)N3[C@@H]1O'                    
 99%|█████████▉| 903/910 [23:57<00:02,  3.08it/s]  